## Import Libraries

In [1]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image

In [10]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

## label the Images and store into respective folders

In [12]:
directory = "Male"
    
if not os.path.exists(directory):
    os.makedirs(directory)

directory1 = "Female"

if not os.path.exists(directory1):
    os.makedirs(directory1)    

directory2 = "Label_Images_Gender"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)     
else:
    os.makedirs(directory2)     
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            new_file_extension = prof['Gender']
            new_file_extension = new_file_extension.title()
            #os.rename(fp, filename+"."+new_file_extension)
            copyfile(filename+".jpg", './Label_Images_Gender/'+ uid + '.' + str(id) + "."+new_file_extension +'.jpg')
            copyfile(filename+".jpg", new_file_extension +'/' + uid + ".jpg")

In [14]:
for file in os.listdir("Label_Images_Gender"):
    file_path = os.path.join("Label_Images_Gender", file)
    try:
        if not file.endswith('.jpg'):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [6]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [7]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    gender =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        gender_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                images.append(resized)
                labels.append(nbr)

                if gender_current == 'Male':
                    gender.append(1)
                else:
                    gender.append(0)

                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(50)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print gender
    
    return images, labels, gender

In [9]:
images, labels, gender = get_images_and_labels('Lable_Images')

cv2.destroyAllWindows()

705
1258
946
1265
191
821
1178
487
1301
588
417
421
1030
200
599
274
70
633
656
591
265
320
731
1250
1125
836
904
955
125
1219
379
1286
565
492
561
118
570
512
264
841
1035
860
648
1136
857
647
804
1293
484
1280
356
662
532
707
721
727
920
116
284
1346
776
326
639
1029
128
161
555
120
1074
945
17
145
1150
493
1285
381
1081
627
803
1311
877
785
485
963
890
137
85
260
434
438
556
817
301
100
1153
115
833
651
989
867
295
581
294
296
980
172
41
1181
363
720
262
1040
935
1215
347
27
956
490
208
616
692
513
1159
1266
1036
255
636
1271
798
1068
206
949
131
258
923
1196
577
1000
1204
547
612
1253
663
1078
535
1169
61
215
732
775
59
165
221
694
1267
1349
50
984
730
1071
176
842
51
1122
747
578
884
928
1348
316
667
967
361
1017
227
425
1056
451
1092
541
575
1290
1008
524
403
1313
834
602
109
518
583
1214
1170
1139
420
181
122
92
43
745
691
220
1114
1238
858
450
604
1305
214
1228
778
188
1182
1275
1232
104
1050
1345
468
1172
67
525
1274
1327
619
365
235
952
14
1164
603
807
286
482
198
851
478
113

In [10]:
images[0].shape

(100, 100)

## Making the Dataset 

In [11]:
res_images = []
res_gender = []

for img in images:
    res_images.append(img.reshape(1,10000))    # (100*100 image size)

for gen in gender:
    res_gender.append(np.array(gen).reshape(1,))
    
res_images = np.concatenate(res_images, axis=0)
res_gender = np.concatenate(res_gender,axis=0)

In [12]:
res_images.shape

(1201, 10000)

In [13]:
res_gender.shape

(1201,)

In [ ]:
importlib

In [14]:
X,y = res_images,res_gender

In [25]:
X_Train, X_Test = X[:1100], X[1100:]
y_Train, y_Test = y[:1100], y[1100:]


print y_Train.shape,y_Test.shape

(1100,) (101,)


In [26]:
# Logistic Regression
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
# load the iris datasets
#dataset = datasets.load_iris()
# fit a logistic regression model to the data
model = LogisticRegression(C=1e5)
model.fit(X_Train,y_Train)
print(model)
# make predictions
expected = y_Test
predicted = model.predict(X_Test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.77      0.74      0.76        50
          1       0.75      0.78      0.77        51

avg / total       0.76      0.76      0.76       101

[[37 13]
 [11 40]]
